In [11]:
import boto3
import os
from config import AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY

# AWS credentials
aws_access_key_id = AWS_ACCESS_KEY_ID
aws_secret_access_key = AWS_SECRET_ACCESS_KEY
region_name = 'us-east-2'  
bucket_name = 'aws-bucket-smart-analytics'

# Local folder path
local_folder_path = './'
# Remote folder. Add "/" at the end
remote_folder =  'SCRIPT/'

# Initialize the S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

print("*********************************UPLOAD*********************************")

# List all the files in the local folder
for root, dirs, files in os.walk(local_folder_path):
    for file in files:
        local_file_path = os.path.join(root, file)
        s3_file_path = os.path.join(remote_folder, os.path.relpath(local_file_path, local_folder_path))
        
        # Upload each file to S3
        s3.upload_file(local_file_path, bucket_name, s3_file_path)
        print(f"FILE  {file}  successfully uploaded to S3 as: {s3_file_path}")

print("")
print("****************************************************************************")
print("*********************************VALIDATION*********************************")
print("")

s3_resource = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)
bucket = s3_resource.Bucket(bucket_name)
success_count = 0  
error_count = 0    
# List all the files in the local folder
for root, dirs, files in os.walk(local_folder_path):
    for file in files:
        local_file_path = os.path.join(root, file)
        s3_file_path = os.path.join(remote_folder, file)
       
        # Check the existence of the object in S3
        object_exists = any(obj.key == s3_file_path for obj in bucket.objects.filter(Prefix=s3_file_path))
        
        if object_exists:
            print(f"The Object {s3_file_path} exists in S3.")

            # Get the size of the local file
            file_size_local = os.path.getsize(local_file_path)

            # Get the size of the file in S3
            s3_object = s3_resource.Object(bucket_name, s3_file_path)
            file_size_s3 = s3_object.content_length

            

            if file_size_local == file_size_s3:
                print(f"The file size of {s3_file_path} in S3 matches the local size.")
                success_count += 1  
            else:
                print(f"ERROR: The file size of {s3_file_path} in S3 does not match the local size.")
                error_count += 1  
        else:
                print(f"ERROR: The object {s3_file_path} is NOT FOUND in S3.")


print(f"Successfully uploaded: {success_count} files")
print(f"Errors encountered: {error_count} files")

*********************************UPLOAD*********************************
FILE  config.py  successfully uploaded to S3 as: SCRIPT/config.py
FILE  create_list_files.ipynb  successfully uploaded to S3 as: SCRIPT/create_list_files.ipynb
FILE  descargar_de_s3.py  successfully uploaded to S3 as: SCRIPT/descargar_de_s3.py
FILE  download_taxi_file.ipynb  successfully uploaded to S3 as: SCRIPT/download_taxi_file.ipynb
FILE  Upload_and_validation.ipynb  successfully uploaded to S3 as: SCRIPT/Upload_and_validation.ipynb
FILE  upload_files.ipynb  successfully uploaded to S3 as: SCRIPT/upload_files.ipynb
FILE  config.cpython-38.pyc  successfully uploaded to S3 as: SCRIPT/__pycache__\config.cpython-38.pyc

****************************************************************************
*********************************VALIDATION*********************************

The Object SCRIPT/config.py exists in S3.
The file size of SCRIPT/config.py in S3 matches the local size.
The Object SCRIPT/create_list_files.i